# Importing packages and data

In [1]:
# Imports
import pandas as pd
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import copy

In [2]:
print(os.getcwd())

/ceph/projects/179_Oncdon/shawn.loo/workspace/notebooks


In [3]:
# Main raw data import
df = pd.read_table("../data/rawdata/all_samples.metaphlan.txt")

# Metadata import
# Importing metadata
metadata_V107_w_inh = pd.read_excel("../data/rawdata/CC_longitudinal_data_standardized_v220250603.xlsx", sheet_name = "CC_longitudinal_metadata_w_inh")
metadata_V107 = pd.read_excel("../data/rawdata/CC_longitudinal_data_standardized_v220250603.xlsx", sheet_name = "CC_longitudinal_metadata")
metadata_V142 = pd.read_excel("../data/rawdata/V350218142_batchC_CC_longitudinal_metadata_dev1_v220250603.xlsx")
metadata_E975_w_inh = pd.read_excel("../data/rawdata/E100051975_BR_longitudinal_metadata_dev1_v220250603.xlsx", sheet_name = "BR_longitudinal_metadata_w_inh")
metadata_E975 = pd.read_excel("../data/rawdata/E100051975_BR_longitudinal_metadata_dev1_v220250603.xlsx", sheet_name = "BR_longitudinal_metadata")

metadata_sets = [metadata_V107, metadata_V142, metadata_E975]

## Separating Data into flow cells

The raw data `data/all_samples.metaphlan.txt` is the summation of all flow cells. We will split and sort them accordingly then apply the filters

In [6]:
# Storing and sorting all_samples into unique flowcell ids
dataset = {}

flowcells = ["V350218107", "V350218142", "E100051975"]

for i in flowcells:
    dataset[i] = pd.concat([df["taxon_name"], df.loc[:, df.columns.str.contains(i)]], axis=1)

In [7]:
print("size of raw main dataset is: ", df.shape)
print("size of raw dataset V107 is: ", dataset["V350218107"].shape)
print("size of raw dataset V142 is: ", dataset["V350218142"].shape)
print("size of raw dataset E975 is: ", dataset["E100051975"].shape)

size of raw main dataset is:  (4624, 440)
size of raw dataset V107 is:  (4624, 302)
size of raw dataset V142 is:  (4624, 89)
size of raw dataset E975 is:  (4624, 51)


## Metadata Duplicate Handling 

There are duplicates in both raw data and metadata.

Before we remove the duplicates in the dataset, we will first see are there any significant difference between them.

We will be using the `sample_description` from the metadata as a determinator column since that duplicates are indicated with _a and _b.

Inspection of duplicates is done via a new dataset just for duplicate and to be plotted for visual analysis of differences.

In [8]:
# Storing each duplicates in samples with separate variables

# For sample V107
meta_V107_a = metadata_V107[metadata_V107["sample_description"].str.endswith("a")]["sample_id"]
meta_V107_b = metadata_V107[metadata_V107["sample_description"].str.endswith("b")]["sample_id"]

# For samaple V142
meta_V142_a = metadata_V142[metadata_V142["sample_description"].str.endswith("a")]["sample_id"]
meta_V142_b = metadata_V142[metadata_V142["sample_description"].str.endswith("b")]["sample_id"]

# Sample E975 have no duplicates
meta_E975 = metadata_E975["sample_id"]

Lets create a function which implies regex to filter our raw data

In [9]:
# Function to retrieve data from all_samples based on metadata samples names
def filterOut_nonMetadata(df, metadata_samples):
    
    # Setting up regex filter
    metadata_samples = '|'.join(metadata_samples)
    
    # Setting up boolean value to filter out from main dataset (masking)
    boolean_mask = pd.Series(df.columns).str.contains(metadata_samples, regex = True)
    
    # Applying filter
    filtered_columns = pd.Series(df.columns)[boolean_mask]
    
    # Return filtered dataset
    return df[filtered_columns].copy()

In [10]:
# Applying function and building a new dataset just for duplicates
# Building dataset by just concatenating different columns

# For V107
V107_dup_A_df = pd.concat([dataset["V350218107"]["taxon_name"], filterOut_nonMetadata(dataset["V350218107"], meta_V107_a)], axis = 1)
V107_dup_B_df = pd.concat([dataset["V350218107"]["taxon_name"], filterOut_nonMetadata(dataset["V350218107"], meta_V107_b)], axis = 1)

# For V142
V142_dup_A_df = pd.concat([dataset["V350218142"]["taxon_name"], filterOut_nonMetadata(dataset["V350218142"], meta_V142_a)], axis = 1)
V142_dup_B_df = pd.concat([dataset["V350218142"]["taxon_name"], filterOut_nonMetadata(dataset["V350218142"], meta_V142_b)], axis = 1)

# For E975
E975_df = pd.concat([dataset["E100051975"]["taxon_name"], filterOut_nonMetadata(dataset["E100051975"], meta_E975)], axis = 1)

duplicated_datasets_lst = [V107_dup_A_df, V107_dup_B_df, V142_dup_A_df, V142_dup_B_df, E975_df]

duplicated_datasets_lst[0]

,taxon_name,179_OncDon__179supA00024a__V350218107_L1_93_,179_OncDon__179supD00322a__V350218107_L2_88_,179_OncDon__179supB00110a__V350218107_L1_59_,179_OncDon__179supD00421a__V350218107_L3_13_,179_OncDon__179supB00008a__V350218107_L4_68_,179_OncDon__179supD00020a__V350218107_L4_81_,179_OncDon__179supA00023a__V350218107_L1_91_,179_OncDon__179supA00035a__V350218107_L2_45_,179_OncDon__179supA00062a__V350218107_L3_96_,...,179_OncDon__179supC00313a__V350218107_L2_70_,179_OncDon__179supA00064a__V350218107_L3_100_,179_OncDon__179supB00007a__V350218107_L4_66_,179_OncDon__179supB00047a__V350218107_L2_103_,179_OncDon__179supA00025a__V350218107_L1_95_,179_OncDon__179supB00010a__V350218107_L4_73_,179_OncDon__179supC00415a__V350218107_L4_99_,179_OncDon__179supC00614a__V350218107_L4_127_,179_OncDon__179supB00039a__V350218107_L2_62_,179_OncDon__179supD00019a__V350218107_L4_79_
0,k__Bacteria,100.00000,100.00000,100.00000,100.00000,100.00000,99.63547,100.00000,100.00000,100.00000,...,100.00000,100.00000,97.81058,100.00000,100.00000,100.00000,100.0,100.0,100.00000,100.00000
1,k__Bacteria|p__Firmicutes,67.88054,63.56849,72.18577,56.38190,53.07202,81.72104,60.41990,59.35715,39.90154,...,44.98734,49.60313,76.63554,29.41244,69.53526,79.81227,100.0,100.0,99.91897,68.45733
2,k__Bacteria|p__Actinobacteria,9.27996,8.61591,3.93038,7.29774,2.91821,3.67028,11.70547,13.52393,10.52356,...,38.85681,12.01365,5.28114,44.47961,15.27623,9.30784,0.0,0.0,0.05682,14.32685
3,k__Bacteria|p__Bacteroidota,18.38777,6.31636,13.15615,17.42360,39.36890,11.65905,25.15347,8.87397,16.48772,...,0.00000,17.45486,10.28786,11.51433,14.13754,5.57742,0.0,0.0,0.01631,13.90139
4,k__Bacteria|p__Proteobacteria,4.45173,21.49924,10.46943,18.89675,0.48339,0.33805,2.72116,18.24495,33.08717,...,16.15586,20.92836,0.77639,14.59362,1.05097,0.15884,0.0,0.0,0.00790,1.00730
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4619,k__Bacteria|p__Actinobacteria|c__Coriobacterii...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.0,0.0,0.00000,0.00000
4620,k__Bacteria|p__Firmicutes|c__Clostridia|o__Eub...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.0,0.0,0.00000,0.00000
4621,k__Bacteria|p__Actinobacteria|c__Actinomycetia...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.0,0.0,0.00000,0.00000
4622,k__Bacteria|p__Actinobacteria|c__Actinomycetia...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.0,0.0,0.00000,0.00000


Lets clean up the format and make more proper names

In [11]:
# Setting taxon_name as index
for i in range(len(duplicated_datasets_lst)):
    
    duplicated_datasets_lst[i] = duplicated_datasets_lst[i].set_index("taxon_name")
    
# Shortening sample name and removing duplicate indicator eg: a and b (the last word in a string)
for df in duplicated_datasets_lst:
    
    # Split by underscores and retrieve unique id
    df.columns = [sample_names.split('_')[3] for sample_names in list(df.columns)]
    
duplicated_datasets_lst[0]

,179supA00024a,179supD00322a,179supB00110a,179supD00421a,179supB00008a,179supD00020a,179supA00023a,179supA00035a,179supA00062a,179supC00513a,...,179supC00313a,179supA00064a,179supB00007a,179supB00047a,179supA00025a,179supB00010a,179supC00415a,179supC00614a,179supB00039a,179supD00019a
taxon_name,,,,,,,,,,,,,,,,,,,,,
k__Bacteria,100.00000,100.00000,100.00000,100.00000,100.00000,99.63547,100.00000,100.00000,100.00000,100.00000,...,100.00000,100.00000,97.81058,100.00000,100.00000,100.00000,100.0,100.0,100.00000,100.00000
k__Bacteria|p__Firmicutes,67.88054,63.56849,72.18577,56.38190,53.07202,81.72104,60.41990,59.35715,39.90154,36.95433,...,44.98734,49.60313,76.63554,29.41244,69.53526,79.81227,100.0,100.0,99.91897,68.45733
k__Bacteria|p__Actinobacteria,9.27996,8.61591,3.93038,7.29774,2.91821,3.67028,11.70547,13.52393,10.52356,46.23864,...,38.85681,12.01365,5.28114,44.47961,15.27623,9.30784,0.0,0.0,0.05682,14.32685
k__Bacteria|p__Bacteroidota,18.38777,6.31636,13.15615,17.42360,39.36890,11.65905,25.15347,8.87397,16.48772,0.00000,...,0.00000,17.45486,10.28786,11.51433,14.13754,5.57742,0.0,0.0,0.01631,13.90139
k__Bacteria|p__Proteobacteria,4.45173,21.49924,10.46943,18.89675,0.48339,0.33805,2.72116,18.24495,33.08717,16.80702,...,16.15586,20.92836,0.77639,14.59362,1.05097,0.15884,0.0,0.0,0.00790,1.00730
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
k__Bacteria|p__Actinobacteria|c__Coriobacteriia|o__Coriobacteriales|f__Coriobacteriaceae|g__Collinsella|s__Collinsella_intestinalis|t__SGB14759,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.0,0.0,0.00000,0.00000
k__Bacteria|p__Firmicutes|c__Clostridia|o__Eubacteriales|f__Clostridiaceae|g__GGB45461|s__GGB45461_SGB63131|t__SGB63131,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.0,0.0,0.00000,0.00000
k__Bacteria|p__Actinobacteria|c__Actinomycetia|o__Actinomycetales|f__Actinomycetaceae|g__Actinomyces|s__Actinomyces_johnsonii|t__SGB15887,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.0,0.0,0.00000,0.00000


In [12]:
# Handling special cases, renaming samples

same_col_df = duplicated_datasets_lst[2]

# Find indexes of duplicate columns
col_indexes = [i for i, col in enumerate(same_col_df.columns) if col == "179fece00032a"]

# Rename them manually using their index
same_col_df.columns.values[col_indexes[0]] = "179fece00032a_1"
same_col_df.columns.values[col_indexes[1]] = "179fece00032a_2"


In [13]:
main_dataset = copy.deepcopy(duplicated_datasets_lst)

# Handling sample ids
Handling sets of samples that starts with FF and removing the last character (eg: the alphabets a or b indicating duplicates)

In [14]:
for i in range(len(duplicated_datasets_lst) - 1):
    
    new_columns = []
    
    for colnames in duplicated_datasets_lst[i].columns:
        
        # Case handling
        if colnames.startswith("FF"):
            new_columns.append(colnames)
            
        elif colnames.startswith("179fece00032a_"):
            new_columns.append(colnames)
        
        else:
            new_columns.append(colnames[:-1])  # remove last character
    
    duplicated_datasets_lst[i].columns = new_columns
    
duplicated_datasets_lst[2]

,179fece00012,179fece000632,179fece00055,179fece000332,179fece00046,179fece000232,179fece00023,179fece00025,179fece00043,179fece00035,...,179fece00062,179fece000432,179fece00044,179fece00032a_2,179fece000132,179fece00063,179fece00064,179fece00045,179fece00024,179fece00042
taxon_name,,,,,,,,,,,,,,,,,,,,,
k__Bacteria,100.00000,100.00000,100.00000,100.00000,100.00000,100.00000,100.00000,100.00000,100.00000,100.00000,...,100.00000,100.00000,100.00000,100.00000,100.00000,100.00000,100.00000,100.00000,100.00000,100.00000
k__Bacteria|p__Firmicutes,65.93411,48.05864,57.53157,99.90979,36.70311,68.87974,99.73688,65.17801,99.94337,62.07111,...,99.99287,55.96109,57.67674,59.98717,76.67538,72.77776,75.30012,43.59935,72.73132,99.99829
k__Bacteria|p__Actinobacteria,19.19386,23.99431,11.91157,0.05799,19.73032,18.19737,0.15161,10.59973,0.01512,5.69612,...,0.00058,19.53379,20.65796,18.20702,15.51041,12.70066,8.39064,11.32860,7.78599,0.00035
k__Bacteria|p__Bacteroidota,7.50906,16.73139,13.32454,0.00197,12.53270,11.65281,0.00159,16.13604,0.03377,13.37346,...,0.00416,14.73487,12.25904,17.44320,0.48255,14.52094,10.36256,17.91829,5.45376,0.00000
k__Bacteria|p__Proteobacteria,6.62733,11.09718,17.23232,0.03025,31.03008,1.26331,0.10992,7.96281,0.00773,18.82067,...,0.00238,9.11311,8.44819,0.84206,7.21344,0.00064,5.94336,27.15376,13.31406,0.00136
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
k__Bacteria|p__Actinobacteria|c__Coriobacteriia|o__Coriobacteriales|f__Coriobacteriaceae|g__Collinsella|s__Collinsella_intestinalis|t__SGB14759,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
k__Bacteria|p__Firmicutes|c__Clostridia|o__Eubacteriales|f__Clostridiaceae|g__GGB45461|s__GGB45461_SGB63131|t__SGB63131,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
k__Bacteria|p__Actinobacteria|c__Actinomycetia|o__Actinomycetales|f__Actinomycetaceae|g__Actinomyces|s__Actinomyces_johnsonii|t__SGB15887,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000


In [15]:
for dataset in duplicated_datasets_lst:
    print(dataset.shape)
    
# Storing E975 from downstream processing
df_E975 = duplicated_datasets_lst.pop()

(4624, 144)
(4624, 146)
(4624, 42)
(4624, 42)
(4624, 48)


In [16]:
# Sorting columns for uniform vectorized operation
duplicated_datasets_lst = [df.sort_index(axis = 1) for df in duplicated_datasets_lst]

# Case Handling for FF samples (the last 6 columns in the first 2 datasets)
FF_samples = [duplicated_datasets_lst[i].iloc[:,-6:] for i in range(2)]

# Dropping them from sorted dataset and grouping them together based on flow cell ID
sup_df = []
fece_df = []

for i in range(4):
    
    # first two samples (107)
    if i <= 1:
        duplicated_datasets_lst[i] = duplicated_datasets_lst[i].drop(columns = FF_samples[i].columns) # dropping FF samples
        sup_df.append(duplicated_datasets_lst[i])
    
    # last two samples (142)
    else:
        fece_df.append(duplicated_datasets_lst[i])